In [4]:
import os
import mlflow
import mlflow.sklearn

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.base import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# silent warnings
import warnings
warnings.filterwarnings("ignore")
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
# import os
# import mlflow
# import pandas as pd
# from datetime import datetime
# import uuid
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.pipeline import Pipeline

# class TrainingSetUp:
#     def __init__(self, *args, **kwargs):
#         self.id = kwargs.get('id', str(uuid.uuid4()))
#         self.created_at = kwargs.get('created_at', datetime.now())
#         self.updated_at = kwargs.get('updated_at', datetime.now())
        
#         if isinstance(self.created_at, str):
#             self.created_at = datetime.strptime(self.created_at, '%Y-%m-%d %H:%M:%S')
        
#         for key, value in kwargs.items():
#             setattr(self, key, value)

#         self.X_train = self.X_test = self.y_train = self.y_test = None
#         self.y_pred = self.y_pred_proba = self.y_pred_proba_train = None
#         self.y_pred_train = self.y_pred_proba_test = self.y_pred_test = None
#         self.__pipeline = self.__model = self.__grid_search = None

#     def load_data(self):
#         self.data = pd.read_csv(self.data_path)
#         self.X = self.data.drop('target', axis=1)
#         self.y = self.data['target']

#     def split_data(self, test_size=0.2, random_state=42):
#         self.X_train, self.X_test, self.y_train, self.y_test = \
#             train_test_split(self.X, self.y, test_size=test_size, random_state=random_state)

#     def pipeline(self):
#         if hasattr(self, 'scalers') and hasattr(self, 'model'):
#             self.__pipeline = Pipeline([
#                 ('scaler', self.scalers),
#                 ('model', self.model)
#             ])
#         else:
#             raise AttributeError('scalers and model must be defined')

#     def grid_search(self):
#         if self.__pipeline is None:
#             self.pipeline()
#         requires = ['param_grid', 'cv', 'scoring']
#         if all(hasattr(self, k) for k in requires):
#             self.__grid_search = GridSearchCV(self.__pipeline,
#                                               self.param_grid,
#                                               cv=self.cv,
#                                               scoring=self.scoring)
#         else:
#             missing = [k for k in requires if not hasattr(self, k)]
#             raise AttributeError(f'{", ".join(missing)} must be defined')

# class TrainModel(TrainingSetUp):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self.__mlflow_uri = None
#         self.__mlflow_metaData = None
#         mlflow_info = kwargs.get('mlflow', {})
#         self.setup_mlflow(mlflow_info.get('exp_name'), mlflow_info.get('artifact_uri'))
#         data_path = kwargs.get('data_path')
#         if data_path:
#             self.load_data(data_path)

#     def setup_mlflow(self, exp_name=None, artifact_uri=None):
#         if exp_name is None:
#             exp_name = f'mlflow_{self.id}'
        
#         os.makedirs("./mlruns", exist_ok=True)
#         default_uri = f'./mlruns/{exp_name}'
#         tracking_uri = os.getenv('MLFLOW_TRACKING_URI_TRAIN', default_uri)
#         mlflow.set_tracking_uri(tracking_uri)
#         mlflow.set_experiment(exp_name)
#         self.__mlflow_uri = tracking_uri

#         if artifact_uri:
#             mlflow.set_artifact_uri(artifact_uri)

#     def train(self):
#         with mlflow.start_run():
#             if self.__grid_search is None:
#                 self.grid_search()
            
#             self.__grid_search.fit(self.X_train, self.y_train)
#             self.__model = self.__grid_search.best_estimator_

#             # Log parameters
#             mlflow.log_params(self.__grid_search.best_params_)

#             # Log metrics
#             mlflow.log_metric("best_score", self.__grid_search.best_score_)

#             # Log model
#             mlflow.sklearn.log_model(self.__model, "model")

#             # Make predictions
#             self.y_pred = self.__model.predict(self.X_test)
#             self.y_pred_proba = self.__model.predict_proba(self.X_test)

#             # Log additional metrics (you can add more as needed)
#             from sklearn.metrics import accuracy_score, f1_score
#             accuracy = accuracy_score(self.y_test, self.y_pred)
#             f1 = f1_score(self.y_test, self.y_pred, average='weighted')
#             mlflow.log_metric("accuracy", accuracy)
#             mlflow.log_metric("f1_score", f1)

#     def evaluate(self):
#         # Add your evaluation logic here
#         pass

#     def save_model(self, path):
#         if self.__model is None:
#             raise Exception("Model has not been trained yet")
#         mlflow.sklearn.save_model(self.__model, path)

#     def load_model(self, path):
#         self.__model = mlflow.sklearn.load_model(path)

# if __name__ == "__main__":
#     # Example usage
#     kwargs = {
#         'mlflow': {'exp_name': 'my_experiment'},
#         'data_path': 'path/to/your/data.csv'
#     }

#     trainer = TrainModel(**kwargs)
#     trainer.load_data()
#     trainer.split_data()
#     trainer.scalers = StandardScaler()
#     trainer.model = RandomForestClassifier()
#     trainer.param_grid = {
#         'model__n_estimators': [100, 200, 300],
#         'model__max_depth': [5, 10, None]
#     }
#     trainer.cv = 5
#     trainer.scoring = 'accuracy'
#     trainer.train()
#     trainer.evaluate()
#     trainer.save_model('path/to/save/model')

In [36]:
import sys
sys.path.append("..")
from utils.model_func import load_config
import logging
from ml.training import TrainModel
from utils.model_func import load_config

CONFIG_PATH = 'config.json'

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# Usage
if __name__ == "__main__":
    logging.info("Starting the training process")

    # Load configuration
    
    config = load_config(CONFIG_PATH)['MODEL1']
    config.update({k: eval(v) if (k == 'scalers' or k=='model') \
                   else v  for k, v in config.items()})

    trainer = TrainModel(config.get('MODEL1'))
    
    trainer.load_data(trainer.data_path)
    trainer.split_data()
    
    from sklearn.preprocessing import StandardScaler
    from sklearn.ensemble import RandomForestClassifier
    
    trainer.scalers = StandardScaler()
    trainer.model = RandomForestClassifier()
    trainer.param_grid = {
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [5, 10, None]
    }
    trainer.cv = 5
    trainer.scoring = 'accuracy'
    
    trainer.train()
    trainer.evaluate()
    trainer.save_model('path/to/save/model')
    
    logging.info("Training process completed")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\lenovo\anaconda3\envs\amsVenv\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_9172\3077490805.py", line 5, in <module>
    from ml.training import TrainModel
  File "e:\amihere\programmingLessons\ALX-Backend\FINAL PROJECT\TripNYC\models\ml\ml.py", line 7, in <module>
    from base.base_model import BaseModel, Base
  File "e:\amihere\programmingLessons\ALX-Backend\FINAL PROJECT\TripNYC\models\ml\..\base\__init__.py", line 3, in <module>
    from models.location.borough import Borough
ModuleNotFoundError: No module named 'models'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lenovo\anaconda3\envs\amsVenv\Lib\site-packages\IPython\core\interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceb

In [34]:
import sys
sys.path.append("..")

from utils.model_func import load_config

config = load_config('../utils/config/ml_config.json')['MODEL1']
print(config)
config.update({k: eval(v) if (k == 'scalers' or k=='model') else v  for k, v in config.items()})
print(config)

{'mlflow': {'exp_name': 'my_experiment'}, 'data_path': 'path/to/your/data.csv', 'scalers': 'StandardScaler()', 'model': 'RandomForestClassifier()', 'param_grid': {'model__n_estimators': [100, 200, 300], 'model__max_depth': [5, 10]}, 'cv': 5, 'scoring': 'accuracy', 'model_path': 'path/to/save/model'}
{'mlflow': {'exp_name': 'my_experiment'}, 'data_path': 'path/to/your/data.csv', 'scalers': StandardScaler(), 'model': RandomForestClassifier(), 'param_grid': {'model__n_estimators': [100, 200, 300], 'model__max_depth': [5, 10]}, 'cv': 5, 'scoring': 'accuracy', 'model_path': 'path/to/save/model'}


In [ ]:
# Create a random dataset
X = np.random.rand(100, 4)
y = np.random.rand(100)

# Define a pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Define a grid search
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [5, 10, None],
    'classifier__min_samples_split': [2, 5, 10],
    '__name__': ['RandomForestClassifier_1']
}

# Create a grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5,
                           scoring='accuracy', n_jobs=-1)

# Set the experiment name
mlflow.set_experiment("RandomForest_GridSearch")
# Set the MLflow tracking URI
tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
if tracking_uri is None:
    os.makedirs("./mlruns", exist_ok=True)
    tracking_uri = "./mlruns"
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))



# Start an MLflow run
with mlflow.start_run():
    # Log parameters
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 5)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Train model (simplified)
    model = RandomForestClassifier(n_estimators=100, max_depth=5)
    model.fit(X_train, y_train)

    # Log metrics
    accuracy = accuracy_score(y_test, model.predict(X_test))
    mlflow.log_metric("accuracy", accuracy)

    # Log model
    mlflow.sklearn.log_model(model, "random_forest_model")

    # Log a plot as an artifact
    plt.figure()
    plt.plot(model.feature_importances_)
    plt.title("Feature Importances")
    plt.savefig("feature_importances.png")
    mlflow.log_artifact("feature_importances.png")

    # Log a tag
    mlflow.set_tag("model_type", "random_forest")

    # Log a custom artifact (e.g., feature names)
    feature_names = ["feature1", "feature2", "feature3"]
    np.savetxt("feature_names.txt", feature_names, fmt='%s')
    mlflow.log_artifact("feature_names.txt")